In [1]:
from __future__ import print_function
import numpy as np
import os
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
from train.shy_provider import *
from models.shy_frustrum_pointnet import *
ROOT_DIR = os.path.abspath('')
OUT_DIR = os.path.join(ROOT_DIR, 'outputs')
TEST_LOG = os.path.join(OUT_DIR, 'test')
TRAIN_LOG = os.path.join(OUT_DIR, 'train')
print(torch.cuda.is_available())

/home/ubuntu/shy-frustrum-pointnets/train /home/ubuntu/shy-frustrum-pointnets
True


In [2]:
EPOCH_CNT = 0
BATCH_SIZE = 32
NUM_POINT = 2048
MAX_EPOCH = 201
BASE_LEARNING_RATE = 0.001
# GPU_INDEX = FLAGS.gpu
# MOMENTUM = FLAGS.momentum
OPTIMIZER = 'adam'
NUM_CHANNEL =  4 # point feature channel
NUM_CLASSES = 2 # segmentation has two classes

In [3]:
# dataset preparation
train_dataset = FrustumDataset(npoints=NUM_POINT, split='train',
        rotate_to_center=True, random_flip=True, random_shift=True, one_hot = True)
test_dataset = FrustumDataset(npoints=NUM_POINT, split='val',
    rotate_to_center=True, one_hot=True)

In [4]:
train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=int(4))

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=int(4))

In [5]:
# creatiting a Frustrum_pointnet
in_dims = 4
net = FrustrumPointNent_v1(in_dims)
if torch.cuda.is_available():
    net.cuda()

In [6]:
# Set an optimizer and a scheduler.
optimizer = torch.optim.Adam(
    net.parameters(), lr=BASE_LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size=20, gamma=0.5)

In [7]:
def run_train(data, net, optimizer, writer=None):
    batch_data, batch_label, batch_center, batch_heading_class, batch_heading_residual, \
    batch_size_class, batch_size_residual, batch_rot_angle, batch_one_hot_vec = data
    
    batch_data = batch_data.float().cuda()
    batch_one_hot_vec = batch_one_hot_vec.float().cuda()
    
    batch_label = batch_label.long()
    batch_center = batch_center.float()
    batch_heading_class = batch_heading_class.long()
    batch_heading_residual = batch_heading_residual.float()
    batch_size_class = batch_size_class.long()
    batch_size_residual = batch_size_residual.float()
    batch_rot_angle = batch_rot_angle.float()
    
    optimizer.zero_grad()
    
    end_points = net.train()(batch_data, batch_one_hot_vec)
    
    loss = get_loss(batch_label, batch_center, batch_heading_class, \
                    batch_heading_residual, batch_size_class, batch_size_residual, end_points, None, None)
    
    # Backprop.
    loss.backward()
    optimizer.step()
    
    iou2ds, iou3ds = compute_box3d_iou(end_points['center'].detach().numpy(), end_points['heading_scores'].detach().numpy(), end_points['heading_residuals'].detach().numpy(), \
                      end_points['size_scores'].detach().numpy(), end_points['size_residuals'].detach().numpy(), \
                      batch_center.numpy(), batch_heading_class.numpy(), batch_heading_residual.numpy(), batch_size_class.numpy(), batch_size_residual.numpy())
    correct = (torch.argmax(end_points['mask_logits'], 2) == batch_label )
    accuracy = torch.sum(correct.float()) / float(BATCH_SIZE*NUM_POINT)
    return loss.detach().numpy(), accuracy.numpy(), np.mean(iou2ds), np.mean(iou3ds)

In [8]:
def run_test(data, net, writer=None):
    batch_data, batch_label, batch_center, batch_heading_class, batch_heading_residual, \
    batch_size_class, batch_size_residual, batch_rot_angle, batch_one_hot_vec = data
    
    batch_data = batch_data.float().cuda()
    batch_one_hot_vec = batch_one_hot_vec.float().cuda()
    
    batch_label = batch_label.long()
    batch_center = batch_center.float()
    batch_heading_class = batch_heading_class.long()
    batch_heading_residual = batch_heading_residual.float()
    batch_size_class = batch_size_class.long()
    batch_size_residual = batch_size_residual.float()
    batch_rot_angle = batch_rot_angle.float()
    
    with torch.no_grad():
        end_points = net.eval()(batch_data, batch_one_hot_vec)

        loss = get_loss(batch_label, batch_center, batch_heading_class, \
                        batch_heading_residual, batch_size_class, batch_size_residual, end_points, None, None)
    
    
        iou2ds, iou3ds = compute_box3d_iou(end_points['center'].detach().numpy(), end_points['heading_scores'].detach().numpy(), end_points['heading_residuals'].detach().numpy(), \
                          end_points['size_scores'].detach().numpy(), end_points['size_residuals'].detach().numpy(), \
                          batch_center.numpy(), batch_heading_class.numpy(), batch_heading_residual.numpy(), batch_size_class.numpy(), batch_size_residual.numpy())
        correct = (torch.argmax(end_points['mask_logits'], 2) == batch_label )
        accuracy = torch.sum(correct.float()) / float(BATCH_SIZE*NUM_POINT)

    return loss.detach().numpy(), accuracy.numpy(), np.mean(iou2ds), np.mean(iou3ds)

In [9]:
def run_epoch_test(dataset, dataloader, train, epoch = None,
        writer=None):
    
    total_loss = 0.0
    total_acc = 0.0
    total_iou2ds = 0.0
    total_iou3ds = 0.0
    n_data = 0
    epoch_str = '' if epoch is None else '[Epoch {}/{}]'.format(
            str(epoch).zfill(len(str(MAX_EPOCH))), MAX_EPOCH)
    
    # Create a progress bar.
    pbar = tqdm(total=n_data, leave=False)
    
    mode = 'Train' if train else 'Test'
    
    
    for i, data in enumerate(dataloader):
        # Run one step.
        loss, seg_accuracy, mean_iou2ds, mean_iou3ds = run_test(data, net, writer) 
        
        

        batch_size = list(data[0].size())[0]
        total_loss += (loss * batch_size)
        total_acc += (seg_accuracy * batch_size)
        total_iou2ds += (mean_iou2ds * batch_size)
        total_iou3ds += (mean_iou3ds * batch_size)
    
        pbar.set_description('{} {} Loss: {:f}, SegAcc : {:.2f}%'.format(
            epoch_str, mode, loss, seg_accuracy))
        pbar.update(batch_size)
        n_data += batch_size

    
    pbar.close()
    mean_loss = total_loss / float(n_data)
    mean_acc = total_acc / float(n_data)
    mean_2d = total_iou2ds/float(n_data)
    mean_3d = total_iou3ds/float(n_data)
    
    if writer!= None:
        step = epoch * 300 + i
        writer.add_scalar('Loss/Test', loss, step)
        writer.add_scalar('SegmentationAccuracy/Test', seg_accuracy, step)
        writer.add_scalar('iou2d/Test', mean_iou2ds, step)
        writer.add_scalar('ioud3d/Test', mean_iou3ds, step)

    return mean_loss, mean_acc, mean_2d, mean_3d
    

def run_epoch_train(dataset, dataloader, train, epoch = None,
        writer=None):
    
    total_loss = 0.0
    total_acc = 0.0
    total_iou2ds = 0.0
    total_iou3ds = 0.0
    n_data = 0
    epoch_str = '' if epoch is None else '[Epoch {}/{}]'.format(
            str(epoch).zfill(len(str(MAX_EPOCH))), MAX_EPOCH)
    
    # Create a progress bar.
    pbar = tqdm(total=n_data, leave=False)
    
    mode = 'Train' if train else 'Test'
    
    
    for i, data in enumerate(dataloader):
        # Run one step.
        loss, seg_accuracy, mean_iou2ds, mean_iou3ds = run_train(data, net, optimizer, writer) 
        
        if train and writer is not None:
            # Write results if training.
            assert(epoch is not None)
            step = epoch * 300 + i
            writer.add_scalar('Loss/Train', loss, step)
            writer.add_scalar('SegmentationAccuracy/Train', seg_accuracy, step)
            writer.add_scalar('iou2d/Train', mean_iou2ds, step)
            writer.add_scalar('ioud3d/Train', mean_iou3ds, step)

        batch_size = list(data[0].size())[0]
        total_loss += (loss * batch_size)
        total_acc += (seg_accuracy * batch_size)
        total_iou2ds += (mean_iou2ds * batch_size)
        total_iou3ds += (mean_iou3ds * batch_size)
    
        pbar.set_description('{} {} Loss: {:f}, SegAcc : {:.2f}%'.format(
            epoch_str, mode, loss, seg_accuracy))
        pbar.update(batch_size)
        n_data += batch_size
        
        # to reduce training we train with 300 batches
        if (i == 300):
            break
    
    pbar.close()
    mean_loss = total_loss / float(n_data)
    mean_acc = total_acc / float(n_data)
    mean_2d = total_iou2ds/float(n_data)
    mean_3d = total_iou3ds/float(n_data)
    return mean_loss, mean_acc, mean_2d, mean_3d
    

    
# def run_epoch_train_and_test(train_dataset, train_dataloader, 
#                              test_dataset, test_dataloader, epoch, writer):
    


In [10]:
# initialize writers
train_writer = SummaryWriter(TRAIN_LOG)
test_writer = SummaryWriter(TEST_LOG)

# training process
for epoch in range(MAX_EPOCH):
    
    # do training
    mean_loss, mean_acc, mean_2d, mean_3d = run_epoch_train(
        train_dataset, train_dataloader, True, epoch, train_writer)
    
    
    
    # do testing
    if (epoch + 1) % 10 == 0:
        # Save the model.
        mean_test_loss, mean_test_acc, mean_test_2d, mean_test_3d = run_epoch_test(
            test_dataset, test_dataloader, False, epoch, test_writer)
        model_file = os.path.join(OUT_DIR, 'model_{:d}.pth'.format(epoch + 1))
        torch.save(net.state_dict(), model_file)
        print("Saved '{}'.".format(model_file))
        epoch_str = '' if epoch is None else '[Epoch {}/{}]'.format(
                str(epoch).zfill(len(str(MAX_EPOCH))), MAX_EPOCH)
        log = epoch_str + ' '
        log += 'Test Loss: {:f}, '.format(mean_test_loss)
        log += 'Test SegAcc: {:f}, '.format(mean_test_acc)
        log += 'Test iou2d, iou3d: {:.2f}%, {:.2f}%, '.format(mean_test_2d, mean_test_3d)
        print(log)
    else:
        epoch_str = '' if epoch is None else '[Epoch {}/{}]'.format(
                str(epoch).zfill(len(str(MAX_EPOCH))), MAX_EPOCH)
        log = epoch_str + ' '
        log += 'Train Loss: {:f}, '.format(mean_loss)
        log += 'Train SegAcc: {:.2f}%, '.format(mean_acc)
        log += 'Train iou2d, iou3d: {:.2f}%, {:.2f}%, '.format(mean_2d, mean_3d)
        print(log)
    
    scheduler.step()

train_writer.close()
test_writer.close()

0it [00:00, ?it/s]                                                                 

[Epoch 000/201] Train Loss: 33.031680, Train SegAcc: 0.71%, Train iou2d, iou3d: 0.13%, 0.10%, 


0it [00:00, ?it/s]                                                                 

[Epoch 001/201] Train Loss: 17.501654, Train SegAcc: 0.81%, Train iou2d, iou3d: 0.18%, 0.14%, 


0it [00:00, ?it/s]                                                                 

...

[Epoch 125/201] Train Loss: 4.909910, Train SegAcc: 0.92%, Train iou2d, iou3d: 0.30%, 0.26%, 


0it [00:00, ?it/s]                                                                 

[Epoch 126/201] Train Loss: 5.001898, Train SegAcc: 0.92%, Train iou2d, iou3d: 0.30%, 0.26%, 


[Epoch 127/201] Train Loss: 4.548826, SegAcc : 0.90%: : 1248it [00:06, 187.35it/s]

KeyboardInterrupt: 